<a href="https://colab.research.google.com/github/aleef-19/rag-chatbot/blob/main/NLP_actual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y transformers accelerate
!pip install -q transformers==4.44.2 accelerate -U

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1


In [ ]:
import transformers
print(transformers.__version__)

4.44.2


In [3]:
!git lfs install
!git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
!git clone https://huggingface.co/deepset/deberta-v3-large-squad2


Git LFS initialized.
fatal: destination path 'all-MiniLM-L6-v2' already exists and is not an empty directory.
fatal: destination path 'deberta-v3-base' already exists and is not an empty directory.
Cloning into 'deberta-v3-large-squad2'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 47 (delta 3), reused 0 (delta 0), pack-reused 37 (from 1)
Unpacking objects: 100% (47/47), 13.69 KiB | 609.00 KiB/s, done.
Filtering content: 100% (4/4), 3.24 GiB | 33.09 MiB/s, done.


In [4]:
from sentence_transformers import SentenceTransformer

# point directly to the folder you just cloned
model1 = SentenceTransformer('/content/all-MiniLM-L6-v2')

print("✅ Model loaded locally")


✅ Model loaded locally


In [5]:
from sentence_transformers import SentenceTransformer, util
import re


def top_kill_sentences_unique(text, kill_words):
    """
    Returns top unique sentences based on similarity to kill_words.

    Args:
        text (str): Input text.
        kill_words (list[str]): List of words to match against (e.g., ['kill', 'neutralize']).
        top_n_sentences (int): Maximum number of unique sentences to return.

    Returns:
        list of tuples: [(sentence, similarity_score), ...] ordered by similarity descending.
    """
    # Split text into sentences and remove duplicates
    sentences = list(set(re.split(r'(?<=[.!?])\s+', text)))

    top_n_sentences=min(5, len(sentences))

    # Encode sentences and kill words
    sentence_embeddings = model1.encode(sentences, convert_to_tensor=True)
    kill_embeddings = model1.encode(kill_words, convert_to_tensor=True)

    # Compute similarity between each sentence and each kill word
    sim_matrix = util.cos_sim(sentence_embeddings, kill_embeddings)

    # Use max similarity per sentence
    max_sim_per_sentence = sim_matrix.max(dim=1).values

    # Pair sentences with their similarity scores
    sentence_score_pairs = list(zip(sentences, max_sim_per_sentence.tolist()))

    # Sort by similarity descending
    sentence_score_pairs.sort(key=lambda x: x[1], reverse=True)

    return sentence_score_pairs[:top_n_sentences]

In [6]:
def top_kill_sentences(text, kill_words):
    """
    Returns top sentences based on word-level similarity to kill_words, keeping unique sentences with highest score.
    """
    # Sentence splitting
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Word tokenization
    words = re.findall(r'\w+', text)

    # Embed words and kill words
    word_embeddings = model1.encode(words, convert_to_tensor=True)
    kill_embeddings = model1.encode(kill_words, convert_to_tensor=True)

    # Compute similarity: each word to all kill words
    sim_matrix = util.cos_sim(word_embeddings, kill_embeddings)

    # Average similarity per word
    avg_sim_per_word = sim_matrix.mean(dim=1)

    top_n_words = min(10, len(words))

    # Get top N words with highest similarity
    top_word_indices = avg_sim_per_word.topk(top_n_words).indices
    top_words = [words[i] for i in top_word_indices]
    top_scores = [avg_sim_per_word[i].item() for i in top_word_indices]

    # Map each word to the first sentence that contains it
    sentence_to_score = {}
    for w, score in zip(top_words, top_scores):
        for sent in sentences:
            if w.lower() in sent.lower():
                # Keep the highest score if sentence already exists
                if sent not in sentence_to_score or score > sentence_to_score[sent]:
                    sentence_to_score[sent] = score
                break

    # Sort sentences by their score descending
    results = sorted(sentence_to_score.items(), key=lambda x: x[1], reverse=True)

    return results


In [7]:
def Ranker(text):
  # Example: two lists from your pipeline
  kill_words = ['kill','eliminate','takedown','neutralize','engage','annihilate','destroy']
  top_sentences_by_sentence = top_kill_sentences_unique(text, kill_words)
  top_sentences_by_word = top_kill_sentences(text, kill_words)

  # Combine scores
  combined_scores = {}

  # Add sentence-level scores
  for sent, score in top_sentences_by_sentence:
      combined_scores[sent] = score

  # Add/merge word-level scores
  for sent, score in top_sentences_by_word:
      if sent in combined_scores:
          # Combine: sum or weighted average
          combined_scores[sent] += score  # sum
          # or: combined_scores[sent] = max(combined_scores[sent], score)
      else:
          combined_scores[sent] = score

  # Sort by combined score descending
  final_ranking = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

  # final_ranking now contains unique sentences with combined score
  return final_ranking


In [8]:
from sentence_transformers import SentenceTransformer, util

descriptions = [
    "Pikachu is a small, yellow electric-type, mouse like Pokémon known for its lightning bolt tail and red cheeks that store electricity.",
    "Charizard is a large,orange fire-breathing dragon-like Pokémon with wings, capable of flying at great speeds.",
    "Bulbasaur is a grass-type, green frog like Pokémon with a plant bulb on its back that grows into a large flower.",
    "Mewtwo is a genetically engineered psychic humanoid Pokémon with immense power, radiation and telepathic abilities."
]

def update_scores_with_similarity(sentences_with_scores, descriptions):
    """
    Takes a list of (sentence, score) tuples and a list of description strings.
    Returns a list of (sentence, updated_score) tuples, sorted in descending
    order by updated_score.
    """

    # Compute description embeddings
    desc_embeddings = model1.encode(descriptions, convert_to_numpy=True)

    # Separate sentences and scores
    sentences = [s for s, _ in sentences_with_scores]
    base_scores = [sc for _, sc in sentences_with_scores]

    # Compute sentence embeddings
    sentence_embeddings = model1.encode(sentences, convert_to_numpy=True)

    # Compute cosine similarity matrix
    similarities = util.cos_sim(sentence_embeddings, desc_embeddings).cpu().numpy()

    # Combine with base scores
    updated = []
    for i, sentence in enumerate(sentences):
        max_sim = similarities[i].max()  # take highest similarity
        updated_score = base_scores[i] + max_sim
        updated.append((sentence, 2*updated_score))

    # Sort in descending order of score
    updated.sort(key=lambda x: x[1], reverse=True)

    return updated


In [14]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Assume 'model1' is defined, e.g., model1 = SentenceTransformer('all-MiniLM-L6-v2')

intent_phrases = [
    "eliminate the target",
    "destroy the creature",
    "engage the hostile",
    "take down the subject",
    "annihilate the enemy"
]

# --- NEW: Define a list of words that signal negation ---
negation_keywords = [
    "not", "don't", "do not", "never", "cease",
    "stand down", "disregard", "ignore", "hold fire"
]

def score_sentences_with_negation_check(sentences_with_scores, intent_phrases, negation_keywords):
    """
    Updates scores based on hostile intent, but penalizes sentences
    that contain negation keywords.
    """

    # Part 1 & 2 remain the same
    sentences = [s for s, _ in sentences_with_scores]
    base_scores = [sc for _, sc in sentences_with_scores]
    sentence_embeddings = model1.encode(sentences, convert_to_numpy=True)
    intent_embeddings = model1.encode(intent_phrases, convert_to_numpy=True)
    intent_similarities = util.cos_sim(sentence_embeddings, intent_embeddings).cpu().numpy()

    # Part 3: Update Scores with the new negation check
    final_results = []
    for i, sentence in enumerate(sentences):
        intent_score = intent_similarities[i].max()

        # --- THIS IS THE KEY CHANGE ---
        # Check if any negation word is in the sentence
        is_negated = any(word in sentence.lower() for word in negation_keywords)

        if is_negated:
            # If a negation word is found, PENALIZE the score
            updated_score = base_scores[i] - intent_score
        else:
            # Otherwise, reward the score as before
            updated_score = base_scores[i] + intent_score

        final_results.append((sentence, updated_score))

    # Part 4: Sort and Return
    final_results.sort(key=lambda x: x[1], reverse=True)

    return [result[0] for result in final_results[:3]]

In [10]:
# ===============================
# 1. Mount Google Drive
# ===============================
from google.colab import drive
drive.mount('/content/drive')

# ===============================
# 2. Imports
# ===============================
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import os

# ===============================
# 3. Load your dataset
# ===============================
df = pd.read_csv('pokemon_orders.csv')  # must have 'sentence' and 'label' columns
sentences = df['sentence'].tolist()
labels = df['label'].tolist()

# Label mappings
id2label = {0: "Pikachu", 1: "Charizard", 2: "Bulbasaur", 3: "Mewtwo", 4: "None"}
label2id = {v: k for k, v in id2label.items()}

# ===============================
# 4. Train/val split
# ===============================
train_texts, val_texts, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

# ===============================
# 5. Tokenizer & Model
# ===============================
local_model_path = "/content/deberta-v3-base"  # path

tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForSequenceClassification.from_pretrained(
    local_model_path,
    num_labels=5,
    id2label=id2label,
    label2id=label2id
)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# ===============================
# 6. Dataset Class
# ===============================
class PokemonDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = PokemonDataset(train_encodings, train_labels)
val_dataset = PokemonDataset(val_encodings, val_labels)

# ===============================
# 7. Training Arguments
# ===============================
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none"
)

# ===============================
# 8. Trainer
# ===============================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# ===============================
# 9. Train Model
# ===============================
trainer.train()

# ===============================
# 10. Save model locally
# ===============================
local_model_dir = "./final_pokemon_model"
trainer.save_model(local_model_dir)
tokenizer.save_pretrained(local_model_dir)

print(f"Model saved locally at {local_model_dir}")

# ===============================
# 11. Save model to Google Drive
# ===============================
drive_model_dir = "/content/drive/MyDrive/final_pokemon_model"
if not os.path.exists(drive_model_dir):
    os.makedirs(drive_model_dir)
trainer.save_model(drive_model_dir)
tokenizer.save_pretrained(drive_model_dir)

print(f"Model also saved to Google Drive at {drive_model_dir}")

# ===============================
# 12. Quick test inference
# ===============================
test_sentence = "take down the humanoid psychic creature at once"
inputs = tokenizer(test_sentence, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
probs = torch.softmax(outputs.logits, dim=-1)
pred_class = torch.argmax(probs).item()
print(f"Predicted class index: {pred_class} → {id2label[pred_class]}")


MessageError: Error: credential propagation was unsuccessful

In [11]:
# Install charset-normalizer (for encoding detection)
!pip install charset-normalizer

import json
from charset_normalizer import from_path

# Step 1: Upload your JSON file
filename = "/content/test_prompts_orders.json"

# Step 2: Detect encoding
detected = from_path(filename).best()
print("Detected encoding:", detected.encoding)

# Step 3: Read file with detected encoding
with open(filename, "r", encoding=detected.encoding) as f:
    text = f.read()

# Step 4: Save back as UTF-8 cleaned file
cleaned_file = "cleaned_prompts.json"
with open(cleaned_file, "w", encoding="utf-8") as f:
    f.write(text)

print(f"File cleaned and saved as {cleaned_file} in UTF-8 ✅")



Detected encoding: cp1250
File cleaned and saved as cleaned_prompts.json in UTF-8 ✅


In [15]:
# Step 5: Load cleaned JSON
with open(cleaned_file, "r", encoding="utf-8") as f:
    data = json.load(f)


# Step 6: Extract prompts
prompts = [item["prompt"] for item in data]

# Step 7: Apply Ranker to each prompt
temp1 = [Ranker(p) for p in prompts]
temp2 = [update_scores_with_similarity(k, descriptions) for k in temp1]
results = [score_sentences_with_negation_check(k, intent_phrases, negation_keywords) for k in temp2]

# Show first few results
for i in range(min(10, len(prompts))):
    print("Prompt:", prompts[i])
    print("Result:", results[i])
    print("-" * 60)

Prompt: HQ REPORT: Situation analysis regarding unusual activity of electric rat in this operational zone. Scouts described sightings of yellow mouse moving in small clusters, often accompanied by subtle disruptions in the environment. Multiple witness statements - low confidence - indicate movement at dawn. Additional activity has been noted from flame dragon groups nearby, though they do not appear hostile at present. Scouts described sightings of rodent of sparks moving in small clusters, often accompanied by subtle disruptions in the environment. Scouts described sightings of yellow mouse moving in small clusters, often accompanied by subtle disruptions in the environment. Small salvageable components can be turned over to engineering. Scouts described sightings of rodent of sparks moving in small clusters, often accompanied by subtle disruptions in the environment. Scouts described sightings of Pikachu moving in small clusters, often accompanied by subtle disruptions in the enviro

In [17]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="deepset/deberta-v3-large-squad2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu


In [20]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("deepset/deberta-v3-large-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/deberta-v3-large-squad2")
print("Model and tokenizer loaded.")

Model and tokenizer loaded.


In [22]:
# 1. Install necessary libraries
!pip install torch

import torch


# Create a question-answering pipeline to handle all the steps for you
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)


# The base context with descriptions of the Pokémon
base_context = """Pikachu is a small, yellow electric mouse like Pokémon known for its lightning bolt tail and red cheeks that store electricity.
Charizard is a large,orange fire-breathing dragon-like Pokémon with wings, capable of flying at great speeds.
Bulbasaur is a grass-type, green frog like Pokémon with a plant bulb on its back that grows into a large flower.
Mewtwo is a genetically engineered psychic humanoid clone Pokémon with immense power, radiation and telepathic abilities."""


# --- Processing Locally ---
final_answers = []

# Loop through each group of three sentences
for sentence_group in results:
    # Join the three sentences in the group into a single string
    sentences_to_add = "\n".join(sentence_group)

    # Use an f-string to combine the base context with the new sentences
    full_context = f"{base_context}\n{sentences_to_add}"

    # The question remains the same
    question = "Which pokemon to kill?"

    # Run the local pipeline
    result = qa_pipeline(question=question, context=full_context)

    answer = result['answer']

    # Print the result for this group and append the answer to our list
    print(f"Added Context:\n'{sentences_to_add}'\nAnswer: {answer}\n---")
    final_answers.append(answer)

# Print the final list of all collected answers
print("\nCollected answers from all groups:")
print(final_answers)

Device set to use cpu


Added Context:
'Order: take down any electric rat encountered and maintain standoff until cleared.
This is a low-resource suppression-do not over-commit assets.
Additional activity has been noted from flame dragon groups nearby, though they do not appear hostile at present.'
Answer:  any electric rat encountered
---
Added Context:
'Engage target: kill the flame dragon and ensure no further escalation.
If you encounter unusual psychic feedback, withdraw immediately.
Additional activity has been noted from Pikachu groups nearby, though they do not appear hostile at present.'
Answer: 
Charizard
---
Added Context:
'Proceed to take down the green seedling found within the sector and send SITREP.
Keep visual on the northern perimeter; movement correlated with wind bursts.
This is a low-resource suppression-do not over-commit assets.'
Answer: 
Mewtwo
---
Added Context:
'Neutralize the winged inferno at first contact, then hold position for assessment.
Do not engage until ROE (Rules of Engagem

In [23]:
# First, make sure you have the library installed:
# pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util

# --- 1. Load the Model (Do this only once in your application) ---
print("Loading the sentence transformer model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded.")

# --- 2. Define Your Knowledge Base (The Pokémon data) ---
pokemon_names = ["Pikachu", "Charizard", "Bulbasaur", "Mewtwo"]
pokemon_descriptions = [
    "A small, yellow, electric-type mouse Pokémon.",
    "A large, orange, fire-breathing dragon-like Pokémon with wings.",
    "A grass-type, green, frog-like Pokémon with a plant bulb on its back.",
    "A genetically engineered psychic humanoid Pokémon with telepathic abilities."
]

# --- 3. The Reusable Function ---
def find_best_match(target_description: str, candidate_names: list, candidate_descriptions: list, model: SentenceTransformer):
    """
    Finds the best match for a target description from a list of candidates using semantic similarity.

    Args:
        target_description (str): The text to find a match for (e.g., "the electric rat").
        candidate_names (list): A list of names for the candidates (e.g., ["Pikachu", ...]).
        candidate_descriptions (list): A list of descriptions for each candidate.
        model (SentenceTransformer): The pre-loaded sentence transformer model.

    Returns:
        str: The name of the candidate with the highest similarity score.
    """
    # Create embeddings for all candidate descriptions
    candidate_embeddings = model.encode(candidate_descriptions, convert_to_tensor=True)

    # Create the embedding for the target description
    target_embedding = model.encode(target_description, convert_to_tensor=True)

    # Calculate cosine similarity between the target and all candidates
    cosine_scores = util.cos_sim(target_embedding, candidate_embeddings)

    # Find the index of the highest score
    best_match_index = cosine_scores.argmax()

    # Return the name of the best matching candidate
    return candidate_names[best_match_index]


# --- 4. Example Usage ---
print("\n--- Running Examples ---")

final = []

for answer in final_answers:
  inferred_pokemon = find_best_match(answer, pokemon_names, pokemon_descriptions, model)
  final.append(inferred_pokemon)

Loading the sentence transformer model...
Model loaded.

--- Running Examples ---


In [ ]:
import torch
import os
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# ===============================
# 1. Mount Google Drive
# ===============================
# This will prompt you for authorization.
drive.mount('/content/drive')

# ===============================
# 2. Define Model Path and Labels
# ===============================
# The exact path where you saved your model in Google Drive.
drive_model_dir = "/content/drive/MyDrive/final_pokemon_model"

# This mapping MUST match the one you used for training.
# 0: Pikachu, 1: Charizard, 2: Bulbasaur, 3: Mewtwo, 4: None
class_mapping = {
    0: "Pikachu",
    1: "Charizard",
    2: "Bulbasaur",
    3: "Mewtwo",
    4: "None"
}


Mounted at /content/drive


In [ ]:
# ===============================
# 3. Load Model and Tokenizer
# ===============================
print(f"Loading model from: {drive_model_dir}")
# Check if the model directory actually exists.
if not os.path.exists(drive_model_dir):
    print("❌ ERROR: Model directory not found!")
    print("Please make sure the path is correct and your training script saved the model successfully.")
else:
    # Load the tokenizer that was saved with the model.
    tokenizer = AutoTokenizer.from_pretrained(drive_model_dir)

    # Load the fine-tuned model weights.
    model = AutoModelForSequenceClassification.from_pretrained(drive_model_dir)

    # Set up the device (use GPU if available, otherwise CPU).
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"✅ Model loaded successfully on device: {device}")

Loading model from: /content/drive/MyDrive/final_pokemon_model
✅ Model loaded successfully on device: cuda


In [ ]:
# ===============================
# 4. Perform Manual Inference
# ===============================


# Set the model to evaluation mode (this disables dropout layers).
model.eval()

# Disable gradient calculations to save memory and speed up inference.
with torch.no_grad():
  for sentence in final:
      # Tokenize the sentence and format it as PyTorch tensors.
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

        # Move the input tensors to the same device as the model (GPU or CPU).
        inputs = {key: val.to(device) for key, val in inputs.items()}

        # Get the model's raw output (logits).
        outputs = model(**inputs)
        logits = outputs.logits

        # Convert logits to probabilities using the softmax function.
        probabilities = F.softmax(logits, dim=-1)

        # Find the class with the highest probability.
        predicted_class_idx = torch.argmax(probabilities, dim=-1).item()

        # Get the name and confidence score for the predicted class.
        predicted_class_name = class_mapping[predicted_class_idx]
        confidence_score = probabilities[0][predicted_class_idx].item()

        # --- Print the results ---
        print("-" * 60)
        print(f"Sentence:           '{sentence}'")
        print(f"Predicted Target:   {predicted_class_name} (Class {predicted_class_idx})")
        print(f"Confidence Score:   {confidence_score:.4f}")

print("-" * 60)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


------------------------------------------------------------
Sentence:           'Order: take down any electric rat encountered and maintain standoff until cleared.'
Predicted Target:   Pikachu (Class 0)
Confidence Score:   0.3811
------------------------------------------------------------
Sentence:           'Engage target: kill the flame dragon and ensure no further escalation.'
Predicted Target:   Charizard (Class 1)
Confidence Score:   0.5530
------------------------------------------------------------
Sentence:           'Proceed to take down the green seedling found within the sector and send SITREP.'
Predicted Target:   Bulbasaur (Class 2)
Confidence Score:   0.6839
------------------------------------------------------------
Sentence:           'Neutralize the winged inferno at first contact, then hold position for assessment.'
Predicted Target:   Charizard (Class 1)
Confidence Score:   0.4432
------------------------------------------------------------
Sentence:           'If